Imports


In [ ]:
!pip install numpy==1.19.3
!pip install opencv-python==4.5.1.48
!pip install tqdm==4.56.0
!pip install mediapipe==0.8.3

In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os
import csv
!pip install mediapipe


In [ ]:
!pip show mediapipe

##Pipeline to preprocess the images to feature vectors containing the landmarks

Locally create a folder named `fitness_poses_images_in` with image samples.

Images should repesent terminal states of desired pose classes. I.e. if you want to classify push-up provide iamges for two classes: when person is up, and when person is down.

There should be about a few hundred samples per class covering different camera angles, environment conditions, body shapes, and exercise variations to build a good classifier.

Required structure of the images_in_folder:
```
fitness_poses_images_in/
  pushups_up/
    image_001.jpg
    image_002.jpg
    ...
  pushups_down/
    image_001.jpg
    image_002.jpg
    ...
  ...
```

Zip the `fitness_poses_images_in` folder:
```
zip -r fitness_poses_images_in.zip fitness_poses_images_in
```

And run the code below to upload it to the Colab runtime

In [ ]:
from google.colab import files
import os
uploaded = files.upload()
os.listdir('.')

In [ ]:
import zipfile
import io

zf = zipfile.ZipFile("/content/drive/MyDrive/fitness_poses_images_in.zip")
zf.extractall()
os.listdir('.')

In [ ]:
import sys

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose


In [ ]:
# Folder with images to use as target poses for classification.
#
# Images should repesent terminal states of desired pose classes. I.e. if you
# want to classify push-up provide iamges for two classes: when person is up,
# and when person is down.
#
# Required structure of the images_in_folder:
#   fitness_poses_images_in/
#     pushups_up/
#       image_001.jpg
#       image_002.jpg
#       ...
#     pushups_down/
#       image_001.jpg
#       image_002.jpg
#       ...
#     ...
images_in_folder = 'fitness_pose_images_in'

# Output folders for bootstrapped images and CSVs. Image will have a predicted
# Pose rendering and can be used to remove unwanted samples.
images_out_folder = 'fitness_pose_images_out'

# Output CSV path to put bootstrapped poses to. This CSV will be used by the
# demo App.
#
# Output CSV format:
#   sample_00001,pose_class_1,x1,y1,z1,x2,y2,z2,...,x33,y33,z33
#   sample_00002,pose_class_2,x1,y1,z1,x2,y2,z2,...,x33,y33,z33
#   ...
#
csv_out_path = 'fitness_pose.csv'

In [ ]:

with open(csv_out_path, 'w') as csv_out_file:
  csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

  # Folder names are used as pose class names.
  pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

  for pose_class_name in pose_class_names:
    print('Bootstrapping ', pose_class_name, file=sys.stderr)

    if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
      os.makedirs(os.path.join(images_out_folder, pose_class_name))

    image_names = sorted([
        n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
        if not n.startswith('.')])
    for image_name in tqdm.tqdm(image_names, position=0):
      try:
        
      # Load image.
        input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)
      except:
        continue
      
      # Initialize fresh pose tracker and run it.
      with mp_pose.Pose(upper_body_only=False) as pose_tracker:
        result = pose_tracker.process(image=input_frame)
        pose_landmarks = result.pose_landmarks
      
      # Save image with pose prediction (if pose was detected).
      output_frame = input_frame.copy()
      if pose_landmarks is not None:
        mp_drawing.draw_landmarks(image=output_frame, landmark_list=pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
      output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
      cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
      
      # Save landmarks.
      if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = output_frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
        csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)

##kNN Classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('fitness_pose.csv')
y = df['Y']
X = df.drop(['Image_name', 'Y'], axis = 1)
classes = {"Y": {"bicep_curls_down": 0, "bicep_curls_up": 2, "Both_Hands_Down": 2, "Both_Hands_Up": 3, "Middle": 4, "One_Hand_Up": 5, "Squatting": 6, "Standing": 7 }}
df = df.replace(classes) 
df.head()
y = df['Y']

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 1)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train_standardized = sc_X.fit_transform(X_train)
x_test_standardized = sc_X.transform(X_test)

In [ ]:
X_test.values


In [ ]:
len(y_test)

In [ ]:
knn = KNeighborsClassifier(n_jobs=-1)
knn_params = {'n_neighbors': [3,4,5,6,7,8,9,10,15,20],
              'weights': ['uniform', 'distance'],
              }
knn_clf = GridSearchCV(knn,knn_params)
print('====> Fitting for SVC model....')
knn_clf.fit(x_train_standardized, y_train)
knn_results = pd.DataFrame(knn_clf.cv_results_)   
knn_results           

In [ ]:
knn_clf.best_params_

In [ ]:
knn_pred = knn_clf.predict(x_test_standardized)

In [ ]:
target_names = ["bicep_curls_down", "bicep_curls_up", "Both_Hands_Down", "Both_Hands_Up", "Middle", "One_Hand_Up", "Squatting", "Standing"] 

In [ ]:
X = scaler.transform(X)
prediction = model.predict(X)


In [ ]:
print(classification_report(y, prediction))
print(f"The knn model is {accuracy_score(prediction,y)*100}% accurate")
print("Confusion Matrix")
confusion_matrix(prediction,y)

In [ ]:
def ExtractFromImage(path):
    image = cv2.imread(path)
    with mp_pose.Pose() as pose:
        result = pose.process(image)
        pose_landmarks = result.pose_landmarks
        if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Map pose landmarks from [0, 1] range to absolute coordinates to get
            # correct aspect ratio.
            frame_height, frame_width = output_frame.shape[:2]
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])
            h = []
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
            h.append(pose_landmarks)   
    return h